In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
_query = """
SELECT
  count(*),
  trim('"' FROM block::text )::int as block
FROM (SELECT
e->'log'->'height'::text as block
from fnode0.missing_messages
 WHERE e->'log'->'event'->4 is not null 
"""

_order = """
 ) as s
  GROUP by s.block
 ORDER BY block ;"""


def query(i):
    return _query + ' AND run = %s ' % i + _order

connection = pg.connect("host='localdb' dbname=load user=load password='load'")

In [6]:
log_runs = pd.read_sql_query('SELECT * from log_runs WHERE id in (3, 1);', con=connection)
log_runs

,id,label,ts
0,1,v5_test_LLLLLLLFF_d100,2019-01-31 17:07:33.116239
1,3,v6_test_LLLLLLLFF_d100,2019-01-31 17:09:05.469478


In [13]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
frames = []
for i, n in log_runs['label'].items():
    r = log_runs['id'][i]
    df = pd.read_sql_query(query(r), con=connection)
    frames.append(df)
    data.append(
        go.Scatter(x=df['block'], y=df['count'], name=n.replace('test_',''))
    )

py.iplot(data, filename='WIP-messages')

In [10]:
s = (frames[0].sum()['count'] / 10, frames[1].sum()['count'] / 10)

100 * (s[0] - s[1]) / s[1] # Average % improvement

8.281975517095812

## Missing Messages

### Test Scenario: 7 Leader 2 followers

* Ran simulator pushing 10 sets of 1000 commit/reveal messages.
* Simulator is configured to drop 10% of network messages
* Network is configured in 'alot' configuration
* Each set is pushed into the API Queue first (without EC funding)
* Then the associated EC address is funded allowing messages to be processed.

### Result: Stability Gains

This data shows that in v6.1.1 ~8% fewer missing messages are being generated indiciating an overall improvement to network efficency. 

This is important for stability under load where the increased overhead of network messages can cause lag and eventually loss of consensus.